In [1]:
#!/usr/bin/env python3

import numpy as np
import cupy as cp
import time

###############################################################################
# 1) RAWKERNEL FOR PLANE INTERSECTION
###############################################################################
plane_triple_intersect = cp.RawKernel(r'''
__device__ __forceinline__ float myAbs(float x) {
    return (x < 0.0f) ? -x : x;
}

__device__ __forceinline__ float myNaN() {
    return __int_as_float(0x7fffffff);  // bitwise pattern for NaN
}

extern "C" __global__
void plane_triple_intersect(
    const float* planes,   // shape (N,4)
    const int* combos,     // shape (M,3)
    float* intersection,   // shape (M,3)
    int M,
    int N
)
{
    int idx = blockDim.x * blockIdx.x + threadIdx.x;
    if (idx >= M) return;

    int i = combos[idx*3 + 0];
    int j = combos[idx*3 + 1];
    int k = combos[idx*3 + 2];

    float A1i = planes[i*4 + 0];
    float A2i = planes[i*4 + 1];
    float A3i = planes[i*4 + 2];
    float Bi  = planes[i*4 + 3];

    float A1j = planes[j*4 + 0];
    float A2j = planes[j*4 + 1];
    float A3j = planes[j*4 + 2];
    float Bj  = planes[j*4 + 3];

    float A1k = planes[k*4 + 0];
    float A2k = planes[k*4 + 1];
    float A3k = planes[k*4 + 2];
    float Bk  = planes[k*4 + 3];

    // determinant
    float det = A1i*(A2j*A3k - A3j*A2k)
              - A2i*(A1j*A3k - A3j*A1k)
              + A3i*(A1j*A2k - A2j*A1k);

    if (myAbs(det) < 1e-12f) {
        float nanval = myNaN();
        intersection[idx*3 + 0] = nanval;
        intersection[idx*3 + 1] = nanval;
        intersection[idx*3 + 2] = nanval;
        return;
    }

    // Solve via Cramer's rule
    float det_x = Bi*(A2j*A3k - A3j*A2k)
                - A2i*(Bj*A3k - A3j*Bk)
                + A3i*(Bj*A2k - A2j*Bk);

    float det_y = A1i*(Bj*A3k - Bk*A3j)
                - Bi*(A1j*A3k - A3j*A1k)
                + A3i*(A1j*Bk  - A3k*Bj);

    float det_z = A1i*(A2j*Bk - Bj*A2k)
                - A2i*(A1j*Bk - Bj*A1k)
                + Bi*(A1j*A2k - A2j*A1k);

    intersection[idx*3 + 0] = det_x / det;
    intersection[idx*3 + 1] = det_y / det;
    intersection[idx*3 + 2] = det_z / det;
}
''', 'plane_triple_intersect')


###############################################################################
# 2) CHECK FEASIBILITY (GPU) -- broadcast approach
###############################################################################
def check_feasibility_gpu(points, planes):
    """
    Given points (Mx3) and planes (Nx4),
    returns bool mask (M,) indicating which points are feasible
    w.r.t. A1*x + A2*y + A3*z <= B for all planes.
    """
    M = points.shape[0]
    N = planes.shape[0]
    if M == 0:
        return cp.zeros((0,), dtype=cp.bool_)

    A1 = planes[:,0].reshape(1,N)
    A2 = planes[:,1].reshape(1,N)
    A3 = planes[:,2].reshape(1,N)
    B  = planes[:,3].reshape(1,N)

    X = points[:,0].reshape(M,1)
    Y = points[:,1].reshape(M,1)
    Z = points[:,2].reshape(M,1)

    lhs = A1*X + A2*Y + A3*Z
    feasible_mask = (lhs <= B + 1e-7).all(axis=1)
    return feasible_mask

###############################################################################
# 3) The incremental 3D LP solver
###############################################################################
def solve_3d_lp_incremental(planes_in, objective, shuffle=True):
    # Convert to cupy
    planes_in = cp.asarray(planes_in, dtype=cp.float32)
    objective = cp.asarray(objective, dtype=cp.float32)

    # Shuffle constraints if desired
    if shuffle:
        host_planes = planes_in.get()
        np.random.shuffle(host_planes)
        planes_in = cp.asarray(host_planes, dtype=cp.float32)

    # -------------------------------------------------------------------------
    # Add bounding planes for x,y,z >= 0, plus large upper bounds
    big = 1e6
    bounding_planes = cp.array([
        # x >= 0 => -x <= 0
        [-1,  0,  0,  0],
        # y >= 0 => -y <= 0
        [ 0, -1,  0,  0],
        # z >= 0 => -z <= 0
        [ 0,  0, -1,  0],
        # x <= 1e6 => x - 1e6 <= 0 => [1,0,0, 1e6]
        [ 1,  0,  0,  big],
        [ 0,  1,  0,  big],
        [ 0,  0,  1,  big],
    ], dtype=cp.float32)

    active_planes = bounding_planes.copy()

    # Build feasible region from bounding planes alone
    feasible_points = _enumerate_feasible_points(active_planes)
    if len(feasible_points) == 0:
        print("No feasible region from bounding planes alone!")
        return cp.array([cp.nan, cp.nan, cp.nan], dtype=cp.float32)

    final_opt = _pick_best(feasible_points, objective)

    # Add user planes one at a time
    for idx in range(len(planes_in)):
        new_plane = planes_in[idx:idx+1,:]  # shape (1,4)

        # Check if final_opt is feasible for that new plane
        lhs = new_plane[0,0]*final_opt[0] + \
              new_plane[0,1]*final_opt[1] + \
              new_plane[0,2]*final_opt[2]

        if lhs <= new_plane[0,3] + 1e-7:
            # still feasible, just record plane
            active_planes = cp.concatenate([active_planes, new_plane], axis=0)
        else:
            # infeasible -> re-enumerate with the new plane
            active_planes = cp.concatenate([active_planes, new_plane], axis=0)
            feasible_points = _enumerate_feasible_points(active_planes)
            if len(feasible_points) == 0:
                print(f"Infeasible after plane #{idx+1}")
                return cp.array([cp.nan, cp.nan, cp.nan], dtype=cp.float32)

            final_opt = _pick_best(feasible_points, objective)

    return final_opt

def _enumerate_feasible_points(planes):
    """
    Collect triple-plane intersections from planes (Nx4).
    Return feasible intersection points (Mx3).
    """
    N = planes.shape[0]
    combos_host = []
    for i in range(N-2):
        for j in range(i+1, N-1):
            for k in range(j+1, N):
                combos_host.append((i,j,k))
    combos_host = np.array(combos_host, dtype=np.int32)
    if len(combos_host) == 0:
        return cp.zeros((0,3), dtype=cp.float32)

    combos_gpu = cp.asarray(combos_host)
    M = combos_gpu.shape[0]
    intersection_out = cp.zeros((M,3), dtype=cp.float32)

    threads_per_block = 128
    blocks_per_grid = (M + threads_per_block - 1) // threads_per_block

    plane_triple_intersect(
        (blocks_per_grid,),
        (threads_per_block,),
        (planes.ravel(), combos_gpu.ravel(),
         intersection_out.ravel(), M, N)
    )

    # Filter out NaN
    not_nan_mask = ~cp.isnan(intersection_out).any(axis=1)
    intersection_candidates = intersection_out[not_nan_mask]

    # Check feasibility
    feasible_mask = check_feasibility_gpu(intersection_candidates, planes)
    feasible_points = intersection_candidates[feasible_mask]
    return feasible_points

def _pick_best(points, objective):
    if len(points) == 0:
        return cp.array([cp.nan, cp.nan, cp.nan], dtype=cp.float32)
    vals = points @ objective
    idx = cp.argmax(vals)
    return points[idx]


###############################################################################
# MAIN: read the text files, solve
###############################################################################
def main():
    # 1) Read LP_A.txt
    with open("LP_A.txt","r") as f:
        lines = f.read().strip().splitlines()
    # First line: "4 3"
    num_constraints, num_vars = map(int, lines[0].split())
    A_rows = []
    for ln in lines[1:]:
        row_parts = ln.split()
        row_floats = list(map(float, row_parts))
        A_rows.append(row_floats)
    A = np.array(A_rows, dtype=np.float32)

    # 2) Read LP_B.txt => right-hand side
    with open("LP_B.txt","r") as f:
        b_vals = f.read().strip().splitlines()
    B = np.array(list(map(float, b_vals)), dtype=np.float32)

    # 3) Combine into (A|B)
    #    for 3D => shape (m,4) => [A1,A2,A3,B]
    planes_in = np.hstack([A, B.reshape(-1,1)])

    # 4) Read objective from LP_C.txt
    with open("LP_C.txt","r") as f:
        c_vals = f.read().strip().splitlines()
    c_floats = list(map(float, c_vals))
    # If more than 3, truncate
    if len(c_floats) > 3:
        c_floats = c_floats[:3]
    objective = np.array(c_floats, dtype=np.float32)

    # 5) Solve
    start = time.time()
    opt = solve_3d_lp_incremental(planes_in, objective, shuffle=False)
    end = time.time()

    print("\n===== RESULTS =====")
    print("Constraints:\n", planes_in)
    print("Objective:", objective)
    print("\nFinal optimum (x,y,z):", opt)

    # Dot product for objective value
    val = float(cp.dot(opt, cp.asarray(objective)))
    print("Objective Value:", val)
    print("Time Elapsed (sec):", end - start)

if __name__ == "__main__":
    main()



===== RESULTS =====
Constraints:
 [[ 1.  0.  0. 10.]
 [ 0.  1.  0. 10.]
 [ 0.  0.  1. 12.]]
Objective: [1. 1. 1.]

Final optimum (x,y,z): [10. 10. 12.]
Objective Value: 32.0
Time Elapsed (sec): 2.7535319328308105
